In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import torchquantum as tq
import random

from torchquantum.datasets import MNIST
from torch.optim.lr_scheduler import CosineAnnealingLR

In [37]:
params=[0.5,0.5]

In [40]:
import torch
import torchquantum as tq

# Define the quantum circuit
circ = tq.(2)

# Apply the CU2 gate to the qubits
theta = torch.tensor(0.5)
phi = torch.tensor(0.25)
circ.apply(tq.gate.CU2Gate(theta, phi), (0, 1))

# Run the circuit on a quantum device
device = tq.QVMDevice()
result = device.run(circ)

# Print the result
print(result)


AttributeError: module 'torchquantum' has no attribute 'qubit'

In [9]:
sq2 = 1/np.sqrt(2)
ch = [[1,0,0,0],[0,sq2,0,sq2],[0,0,1,0],[0,sq2,0,-sq2]]
q_dev = tq.QuantumDevice(n_wires=3)
q_dev.apply(tq.RX(has_params=True,  init_params= 0.5,wires=[0]))
q_dev.apply(tq.QubitUnitary(
   has_params=True,init_params=([[1,0,0,0],[0,sq2,0,sq2],[0,0,1,0],[0,sq2,0,-sq2]]),wires=[0]))
k =[]
for i in range(3):
   k.append(tq.expval(q_dev,wires=i, observables= tq.PauliZ(wires=i)))
#b = tq.MeasureAll(tq.PauliZ)
#print(b(q_dev))
#print(a1)
k = torch.tensor([k])
k

tensor([[0.8776, 1.0000, 1.0000]])

In [172]:
bsz = 10
k.view(bsz, 3)

RuntimeError: shape '[10, 3]' is invalid for input of size 3

In [153]:
b(q_dev).type

<function Tensor.type>

In [154]:
k.type

<function Tensor.type>

In [ ]:
[tensor([[0.8776]]), tensor([[1.]]), tensor([[1.]])]

In [171]:
b(q_dev)[0][:]

tensor([0.8776, 1.0000, 1.0000])

In [112]:
print(b(q_dev).shape)
print(k.shape)

torch.Size([1, 3])
torch.Size([1, 3])


In [7]:

def controlled_H(qdev, target,control):
      qdev.apply(tq.QubitUnitary(
      has_params=True,init_params=([[1,0,0,0],[0,sq2,0,sq2],[0,0,1,0],[0,sq2,0,-sq2]]),wires=[target,control]))
     
q_dev = tq.QuantumDevice(n_wires=2)
q_dev.apply(tq.PauliX(wires = 0))
q_dev.apply(tq.PauliX(wires = 1))
controlled_H(q_dev,1,0)
#q_dev.apply(tq.QubitUnitary(
#   has_params=True,init_params=(ch),wires=[1,0])) # wires = [target, control]
print(q_dev)

NameError: name 'sq2' is not defined

In [2]:
class QuanvolutionFilter(tq.QuantumModule):
  # the __init__ method initializes the quantum device, the general encoder,
  # a random quantum layer, and a measurement operator.
    def __init__(self):
        super().__init__()
        self.n_wires = 4 + 2 
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        # encoding the input data
        self.encoder = tq.GeneralEncoder(
        [   {'input_idx': [0], 'func': 'ry', 'wires': [0]},
            {'input_idx': [1], 'func': 'ry', 'wires': [1]},
            {'input_idx': [2], 'func': 'ry', 'wires': [2]},
            {'input_idx': [3], 'func': 'ry', 'wires': [3]},])
        


        # random circuit layer 
        self.q_layer = tq.RandomLayer(n_ops=8, wires=list(range(self.n_wires)))
        self.measure = tq.MeasureAll(tq.PauliZ)
        #self.expval = tq.expval()
    
# x has the dimension of (batch_size, 28, 28) representing a batch of greyscale images
#The method first reshapes the input data into a 2D array of shape 
#(batch_size, 784) by concatenating adjacent 2x2 blocks of pixels.
# data is the new reshaped tensor 
    def forward(self, x, use_qiskit=False):
        bsz = x.shape[0] # batch size
        size = 28 # height and width of an image
        x = x.view(bsz, size, size) # view all data 

        data_list = []

        for c in range(0, size, 2):
            for r in range(0, size, 2):
                data = torch.transpose(torch.cat((x[:, c, r], x[:, c, r+1], x[:, c+1, r], x[:, c+1, r+1])).view(4, bsz), 0, 1)
                if use_qiskit:
                    data = self.qiskit_processor.process_parameterized(
                        self.q_device, self.encoder, self.q_layer, self.measure, data)
                else:
                    self.encoder(self.q_device, data)
                    
                    #haar wavelet
                    # level 1 
                    self.q_device.h(wires = 3) 
                    self.q_device.swap([3,2])
                    self.q_device.swap([2,1])
                    self.q_device.swap([1,0])

                    # level 2 
                    controlled_H(self.q_device, target=2, control= 3)
                    self.q_device.cswap([3,2,1])
                    self.q_device.cswap([3,1,0])

                    # level 3
                    self.q_device.ccx([2,3,4])
                    controlled_H(self.q_device, target=1, control= 4)
                    self.q_device.ccx([2,3,4])
                    #perm
                    self.q_device.ccx([2,3,4])
                    self.q_device.cswap([4,1,0])
                    self.q_device.ccx([2,3,4])

                    #level 4
                    self.q_device.ccx([2,3,4])
                    self.q_device.ccx([1,4,5])
                    controlled_H(self.q_device, target=0, control= 5)
                    self.q_device.ccx([1,4,5])
                    self.q_device.ccx([2,3,4])



                    self.q_layer(self.q_device)
                    data = self.measure(self.q_device)
                    #data = []
                    #for i in range(4):
                    #    data.append(tq.expval(self.q_device,wires=i, observables= tq.PauliZ(wires=i)))

                    #data = torch.tensor([data])

                    
                data_list.append(data[0][:4].view(bsz, 4)) # only keep the first 4 qubits
        
        result = torch.cat(data_list, dim=1).float()
        
        return result


In [12]:
QuanvolutionFilter()

QuanvolutionFilter(
  (q_device):  class: QuantumDevice 
   device name: default 
   number of qubits: 6 
   batch size: 1 
   current computing device: cpu 
   current states: [[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
    0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
    0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
    0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
    0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
    0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
    0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
  (encoder): GeneralEncoder()
  (q_layer): RandomLayer(
    (op_list): QuantumModuleList(
      (0):  class: RY 
       parameters: Parameter containing:
      tensor([[1.1869]], requires_grad=True) 
       wires: [4] 
       inverse: False
      (1):  class: RY 
       parameters: Parameter containing:
      tensor([[-0.0637]], requires_grad=Tr

In [15]:
torch.nn.Linear?

Init signature:
torch.nn.Linear(
    in_features: int,
    out_features: int,
    bias: bool = True,
    device=None,
    dtype=None,
) -> None
Docstring:     
Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

On certain ROCm devices, when using float16 inputs this module will use :ref:`different precision<fp16_on_mi200>` for backward.

Args:
    in_features: size of each input sample
    out_features: size of each output sample
    bias: If set to ``False``, the layer will not learn an additive bias.
        Default: ``True``

Shape:
    - Input: :math:`(*, H_{in})` where :math:`*` means any number of
      dimensions including none and :math:`H_{in} = \text{in\_features}`.
    - Output: :math:`(*, H_{out})` where all but the last dimension
      are the same shape as the input and :math:`H_{out} = \text{out\_features}`.

Attributes:
    weight: the learnable weights of the module of shape
        :m

In [3]:

class HybridModel(torch.nn.Module): # nn.Module is the base class for all neural network modules
    def __init__(self): # inherit from nn.Module
        super().__init__() # super calls the initialization method of the parent class nn.module
        self.qf = QuanvolutionFilter()
        self.linear = torch.nn.Linear(4*14*14, 10) # self.linear is a linear layer with 10 output units, outputting the logits for each class
    # here the input size is 4*14*14 because the input is reshaped into 4*14*14 after the quanvolution filter
    # 10 is the number of classes. classes are u
    def forward(self, x, use_qiskit=False):
        with torch.no_grad():
          x = self.qf(x, use_qiskit)
        x = self.linear(x)
        return F.log_softmax(x, -1) # log_softmax is the activation function for the output layer

class HybridModel_without_qf(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(28*28, 10)
    
    def forward(self, x, use_qiskit=False):
        x = x.view(-1, 28*28)
        x = self.linear(x)
        return F.log_softmax(x, -1)

In [22]:
model = HybridModel()
x = torch.randn(1,784)  # 64 is the batch size
print(model(x))

tensor([[-2.2454, -2.3707, -2.3760, -2.2943, -2.3218, -2.3706, -1.9413, -2.5021,
         -2.0558, -2.7807]], grad_fn=<LogSoftmaxBackward0>)


In [20]:
MNIST?

Init signature:
MNIST(
    root: str,
    train_valid_split_ratio: List[float],
    center_crop=28,
    resize=28,
    resize_mode='bilinear',
    binarize=False,
    binarize_threshold=0.1307,
    digits_of_interest=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
    n_test_samples=None,
    n_valid_samples=None,
    fashion=False,
    n_train_samples=None,
)
Docstring:     
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)
File:           ~/.local/lib/python3.8/site-packages/torchquantum/datasets/mnist.py
Type:           type
Subclasses:     


In [14]:
glioma = np.load('/home/iisers/Documents/MRI_dataset/glioma_tumor/batch_0.npy')

In [37]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from skimage import io
import torchvision.transforms as transforms


In [35]:
pip install scikit-image

     |████████████████████████████████| 13.4 MB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 218 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 12.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [32]:
class Oral_Can_Data(Dataset):
    def __init__(self, csv_file, root_dir,transform = None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations) # returns the number of samples in the dataset
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        if self.transform:
            image = self.transform(image)
        
        return (image, y_label) # returns the image and the label


In [40]:
dataset = Oral_Can_Data(csv_file='/home/iisers/Documents/oral cancer project/labels.csv',root_dir='/home/iisers/Documents/oral cancer project/Combined_data', transform = transforms.ToTensor())

FileNotFoundError: [Errno 2] No such file or directory: '/home/iisers/Documents/oral cancer project/labels.csv'

In [4]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
dataset = MNIST(
    root='./mnist_data',
    train_valid_split_ratio=[0.9, 0.1],
    n_test_samples=30,
    n_train_samples=50,
)
dataflow = dict()

for split in dataset:
    sampler = torch.utils.data.RandomSampler(dataset[split])
    dataflow[split] = torch.utils.data.DataLoader(
        dataset[split],
        batch_size=1,
        sampler=sampler,
        num_workers=6,
        pin_memory=True)

100%|██████████| 9912422/9912422 [00:12<00:00, 818016.46it/s] 


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 2434398.99it/s]


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 1807160.55it/s]


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 2128075.15it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw




[2023-03-16 13:10:30.311] Only use the front 50 images as TRAIN set.
[2023-03-16 13:10:30.406] Only use the front 30 images as TEST set.


In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = HybridModel().to(device) # model is a HybridModel
model_without_qf = HybridModel_without_qf().to(device)
n_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)

/home/iisers/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [10]:
accu_list1 = []
loss_list1 = []
accu_list2 = []
loss_list2 = []

def train(dataflow, model, device, optimizer):
    for feed_dict in dataflow['train']:
        inputs = feed_dict['image'].to(device)
        targets = feed_dict['digit'].to(device)

        outputs = model(inputs)
        loss = F.nll_loss(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"loss: {loss.item()}", end='\r')


def valid_test(dataflow, split, model, device, qiskit=False):
    target_all = []
    output_all = []
    with torch.no_grad():
        for feed_dict in dataflow[split]:
            inputs = feed_dict['image'].to(device)
            targets = feed_dict['digit'].to(device)

            outputs = model(inputs, use_qiskit=qiskit)

            target_all.append(targets)
            output_all.append(outputs)
        target_all = torch.cat(target_all, dim=0)
        output_all = torch.cat(output_all, dim=0)

    _, indices = output_all.topk(1, dim=1)
    masks = indices.eq(target_all.view(-1, 1).expand_as(indices))
    size = target_all.shape[0]
    corrects = masks.sum().item()
    accuracy = corrects / size
    loss = F.nll_loss(output_all, target_all).item()

    print(f"{split} set accuracy: {accuracy}")
    print(f"{split} set loss: {loss}")

    return accuracy, loss

for epoch in range(1, n_epochs + 1):
    # train
    print(f"Epoch {epoch}:")
    train(dataflow, model, device, optimizer)
    print(optimizer.param_groups[0]['lr'])

    # valid
    accu, loss = valid_test(dataflow, 'test', model, device, )
    accu_list1.append(accu)
    loss_list1.append(loss)
    scheduler.step()


Epoch 1:
0.005 3.8555865287780762
test set accuracy: 0.26666666666666666
test set loss: 1.929352879524231
Epoch 2:
0.004522542485937368540226
test set accuracy: 0.5333333333333333
test set loss: 1.4241527318954468
Epoch 3:
0.003272542485937368317686
test set accuracy: 0.4666666666666667
test set loss: 1.3936468362808228
Epoch 4:
0.0017274575140626314981266
test set accuracy: 0.5666666666666667
test set loss: 1.446394681930542
Epoch 5:
0.0004774575140626316650633
test set accuracy: 0.7333333333333333
test set loss: 1.223548173904419
